In [ ]:
import os
new_directory = '/home/franciscoperez/Documents/GitHub/CNN-PELSVAE2/cnn-pels-vae/'
os.chdir(new_directory)
import src.utils as utils
import pickle 
import numpy as np 
from concurrent.futures import ThreadPoolExecutor
import yaml 
from collections import Counter
import pandas as pd 

with open('src/configuration/paths.yaml', 'r') as file:
    YAML_FILE = yaml.safe_load(file)

PATHS =YAML_FILE['paths']

PATH_MODELS: str = PATHS["PATH_MODELS"]


def read_light_curve_ogle(example_train, values_count, lenght_lc=100, sn_ratio = 2):
    
    numpy_array_lcus_test = np.empty((0, 0, 2))
    numpy_array_lcus_train =  np.empty((0, 0, 2))
    numpy_y_test =  np.empty((0, ), dtype=object)
    numpy_y_train = np.empty((0, ), dtype=object)
    
    def task():
        return utils.insert_lc(example_train, numpy_array_lcus_train,
                         numpy_y_train, values_count=values_count, 
                         signal_noise=sn_ratio, 
                         lenght_lc=lenght_lc, train_set=True, file_name='train')
    
    with ThreadPoolExecutor() as executor:
        future = executor.submit(task)
        
        numpy_array_lcus_train, numpy_y_train = future.result()
    
    return numpy_array_lcus_train, numpy_y_train

results = []
for seq_length in [100, 200, 300]:
    for sn in [2, 4, 6]:
        _ , id_train, values_count  = utils.get_ids(n=500000) 
        x_train, y_train  = read_light_curve_ogle(id_train, values_count, lenght_lc = seq_length, sn_ratio = sn)
                
        print('Loaded shape training set: ', x_train.shape)
        if int(x_train.shape[1])!=2:
            x_train = x_train.transpose(0, 2, 1)
            
        print('Modified shape training set: ', x_train.shape)

        with open(PATH_MODELS+'label_encoder_vae.pkl', 'rb') as f:
                label_encoder = pickle.load(f)
        results.append([seq_length, sn,  Counter(y_train)])
        print(Counter(y_train))
        pd.DataFrame(results).to_csv('results.csv')

LPV      321451
ECL       35061
RRLYR     28847
CEP        4093
DSCT       1496
T2CEP       367
DPV         106
ACEP         36
RCB          20
DN           17
ACV           5
WD            1
YSO           1
Name: class, dtype: int64
LPV      321451
ECL       35061
RRLYR     28847
CEP        4093
DSCT       1496
T2CEP       367
DPV         106
ACEP         36
RCB          20
Name: class, dtype: int64
LPV      321451
ECL       35061
RRLYR     28847
CEP        4093
DSCT       1496
T2CEP       367
DPV         106
ACEP         36
RCB          20
Name: class, dtype: int64
clases:  ['LPV' 'RRLYR' 'ECL' 'DSCT' 'CEP']
clases:  [0 1 2 3 4]


Processing Light Curves: 100%|███████████████████████████████████████████████████| 390948/390948 [5:47:57<00:00, 18.73it/s]


shape:  (281670, 100, 2) (281670,)
Loaded shape training set:  (281670, 100, 2)
Modified shape training set:  (281670, 2, 100)
Counter({'LPV': 216602, 'ECL': 34646, 'RRLYR': 25423, 'CEP': 3530, 'DSCT': 1469})
LPV      321451
ECL       35061
RRLYR     28847
CEP        4093
DSCT       1496
T2CEP       367
DPV         106
ACEP         36
RCB          20
DN           17
ACV           5
WD            1
YSO           1
Name: class, dtype: int64
LPV      321451
ECL       35061
RRLYR     28847
CEP        4093
DSCT       1496
T2CEP       367
DPV         106
ACEP         36
RCB          20
Name: class, dtype: int64
LPV      321451
ECL       35061
RRLYR     28847
CEP        4093
DSCT       1496
T2CEP       367
DPV         106
ACEP         36
RCB          20
Name: class, dtype: int64
clases:  ['LPV' 'RRLYR' 'ECL' 'DSCT' 'CEP']
clases:  [0 1 2 3 4]


Processing Light Curves:  69%|█████████████████████████████████▊               | 269715/390948 [2:58:37<2:11:12, 15.40it/s]